In [ ]:
import pandas as pd ; import numpy as np
import os , time, gc, random
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter('ignore')


In [ ]:
%%time
train = pd.read_csv('../input/tabular-playground-series-sep-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-sep-2021/test.csv')
ss = pd.read_csv('../input/tabular-playground-series-sep-2021/sample_solution.csv')


In [ ]:
%%time
features = [col for col in test.columns if 'f' in col]
TARGET = 'claim'
target = train[TARGET].copy()
train['n_missing'] = train[features].isna().sum(axis=1)
test['n_missing'] = test[features].isna().sum(axis=1)

train['std'] = train[features].std(axis=1)
test['std'] = test[features].std(axis=1)

features += ['n_missing', 'std']
n_missing = train['n_missing'].copy()
train[features] = train[features].fillna(train[features].mean())
test[features] = test[features].fillna(test[features].mean())
scaler = RobustScaler()
train[features] = scaler.fit_transform(train[features])
test[features] = scaler.transform(test[features])


In [ ]:
lgb_params = {
    'objective': 'binary',
    'n_estimators': 10000,
    'random_state': 42,
    'learning_rate': 4e-3,
    'subsample': 0.6,
    'subsample_freq': 1,
    'colsample_bytree': 0.4,
    'reg_alpha': 12.0,
    'reg_lambda': 1e-1,
    'min_child_weight': 256,
    'min_child_samples': 20,
}

In [ ]:
from sklearn.model_selection import train_test_split
lgb_oof = np.zeros(train.shape[0])
lgb_pred = np.zeros(test.shape[0])
lgb_importances = pd.DataFrame()

x_train, x_valid, y_train, y_valid = train_test_split(train[features], target, test_size = 0.2, shuffle=True, random_state=42)


    
start = time.time()
model = lgb.LGBMClassifier(**lgb_params)
model.fit(
    x_train, 
    y_train,
    eval_set=[(x_valid, y_valid)],
    eval_metric='auc',
    early_stopping_rounds=200,
    verbose=1000,
)
lgb_pred += model.predict_proba(test[features])[:, -1] 



In [ ]:
ss[TARGET] = lgb_pred
ss.to_csv("submission.csv", index=False)
